<a href="https://colab.research.google.com/github/savindumahasen/AI-Detection-System-/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
# install the necessary libraries
!pip install pandas
!pip install numpy
!pip install opendatasets

In [57]:
# import necessary libraries
import pandas as pd
import numpy as np
import opendatasets as od
import csv

In [58]:
## download the dataset
od.download("https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text")

Skipping, found downloaded files in "./ai-vs-human-text" (use force=True to force download)


In [74]:
## load the datast and read first five records
import csv

dataset = pd.read_excel("/content/Dataset.xlsx")

dataset.head()

,sr.no,text,source,label_name,label_id
0,0,"Produced by Chris Young Ritzen, Tamara Weitzma...",openwebtext,human-written,1
1,1,"Lawrence Eugene Doby (December 13, 1923 – June...",openwebtext,human-written,1
2,2,Shock and Awe\n\nGabe Newell is about to make ...,openwebtext,human-written,1
3,3,The Best Answers to Tough Interview Questions\...,openwebtext,human-written,1
4,4,Contents:\n\n1.) Preamble\n\n2.) Understand Yo...,openwebtext,human-written,1


In [75]:
## get the information  of dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11580 entries, 0 to 11579
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sr.no       11580 non-null  int64 
 1   text        11580 non-null  object
 2   source      11580 non-null  object
 3   label_name  11580 non-null  object
 4   label_id    11580 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 452.5+ KB


In [76]:
## length of the dataset
len(dataset)

11580

In [78]:
# get the value counts of label_id
dataset['label_id'].value_counts()

,count
label_id,
1,5790
0,5790
